# Seminar on Graphs for NLP: Vector representations

## Plan for today:

#### 0. What a taxonomy is. Taxonomy Enrichment task.
#### 1. Node2vec model. Implementation of node2vec.
#### 2. Embedding generation for the OOV words for node2vec. Linear transformation model.
#### 3. Graph Neural networks: GCN and GAT
#### 4. GraphBERT: Only Attention is Needed for Learning Graph Representations

# 0. Taxonomy

A taxonomy is a hierarchical structure of units in terms if class inclusion such that superordinate units in the hierarchy include, or subsume, all items in subordinate units. Taxonomies are typically represented as having tree structures.

![](https://www.digital-mr.com/media/cache/51/6f/516f493d37a7b4895f678843b6383e48.png)


Taxonomies can be represented as graphs!

Let us download the most popular and well-known taxonomy called WordNet. You may also use the `from nltk.corpus import wordnet as wn`, but keep in mind that you can operate with earlier versions.

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.2 MB/s eta 0:00:00


In [3]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!curl -L -o 'wordnet_n_is_directed_12_en_synsets.zip' 'https://drive.google.com/u/0/uc?id=1TvWsvz8UC0RPKHBx2GRi-iChVG4oTz-m&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  206M  100  206M    0     0  34.0M      0  0:00:06  0:00:06 --:--:-- 46.8M


In [5]:
!unzip wordnet_n_is_directed_12_en_synsets.zip

Archive:  wordnet_n_is_directed_12_en_synsets.zip
   creating: wordnet_n_is_directed_1_en_synsets/
  inflating: wordnet_n_is_directed_1_en_synsets/link  
   creating: wordnet_n_is_directed_1_en_synsets/.ipynb_checkpoints/
  inflating: wordnet_n_is_directed_1_en_synsets/.ipynb_checkpoints/link-checkpoint  
  inflating: wordnet_n_is_directed_1_en_synsets/node  


In [6]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
from gensim.models.poincare import PoincareModel
import numpy as np
import time
import os

In [8]:
from nltk.corpus import wordnet as wn

In [9]:
wn.synset("guy.n.01").hyponyms()[0].definition()

'an informal British term for a youth or man'

In [10]:
path = f"wordnet_n_is_directed_1_en_synsets/"

link_path = os.path.join(path, "link")
node_path = os.path.join(path, "node")

In [11]:
id2synset = {}
fasttext_dict = {}

with open(node_path) as f:
    for line in f:
        line_split = line.split("\t")
        id2synset[line_split[0].strip()] = line_split[-1].strip()
        fasttext_dict[line_split[-1].strip()] = np.array([float(num) for num in line_split[1:-1]])

In [12]:
link_pairs = set()
with open(link_path) as f:
    for line in f:
        line_split = line.split("\t")
        link_pairs.add((id2synset[line_split[0].strip()], id2synset[line_split[-1].strip()]))

# 4. Graph Neural Networks

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

In [15]:
import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.utils import train_test_split_edges
import torch_geometric.transforms as T
from torch_geometric.data import Data

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


## Data preparation

In [16]:
from gensim.models.keyedvectors import KeyedVectors

fasttext = KeyedVectors(vector_size=300)
fasttext.add_vectors(list(fasttext_dict.keys()), list(fasttext_dict.values()))

In [17]:
import networkx as nx

In [18]:
G = nx.DiGraph()

for pair in link_pairs:
    G.add_edge(*pair)

In [19]:
for pair in link_pairs:
    print(pair)
    break

('flight.n.09', 'connecting_flight.n.01')


In [20]:
def create_edge_list(G):
    starts = []
    ends = []
    for left, right in G.edges:
        if left in fasttext.key_to_index and right in fasttext.key_to_index:
            starts.append(fasttext.key_to_index[left])
            ends.append(fasttext.key_to_index[right])
    return torch.tensor([starts, ends], dtype=torch.long)

In [21]:
index_to_key = dict(map(reversed, fasttext.key_to_index.items()))

In [22]:
edge_index = create_edge_list(G)

In [23]:
edge_index.shape

torch.Size([2, 72370])

In [24]:
x = torch.tensor([fasttext[index_to_key[int(i)]] for i in index_to_key], dtype=torch.float)

<ipython-input-24-ceafeacad9cc>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  x = torch.tensor([fasttext[index_to_key[int(i)]] for i in index_to_key], dtype=torch.float)


In [25]:
x.shape

torch.Size([78748, 300])

In [26]:
data = Data(x=x, edge_index=edge_index)
#data = train_test_split_edges(data)

In [27]:
from torch_geometric.transforms import RandomLinkSplit

In [28]:
transform = RandomLinkSplit(is_undirected=True, split_labels=True)
train_data, val_data, test_data = transform(data)

### GCN and GAT Encoder

The following code snippet describes the Encoder module with GCN or GAT networks.

In [29]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, mode="gcn"):
        super(Encoder, self).__init__()
        if mode == "gcn":
            self.conv1 = pyg_nn.GCNConv(in_channels, 2 * out_channels, cached=True)
            self.conv2 = pyg_nn.GCNConv(2 * out_channels, out_channels, cached=True)
        elif mode == 'gat':
            self.conv1 = pyg_nn.GATConv(in_channels, 2 * out_channels)
            self.conv2 = pyg_nn.GATConv(2 * out_channels, out_channels)
        else:
            raise Exception("Encoder mode is not recognized, try gcn/gat")

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

def train(epoch):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    writer.add_scalar("loss", loss.item(), epoch)
    return loss.item()

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [30]:
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

channels = 64
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA availability:', torch.cuda.is_available())

CUDA availability: True


## Variational Graph Auto-Encoders

https://arxiv.org/pdf/1611.07308.pdf

The pipeline is working as follows: first, we train a graph autoencoder with GCN or GAT under the hoot. During the evaluation phase, the latent representations of the autoencoder are actually the embeddings we are looking for.

In [31]:
model = pyg_nn.GAE(Encoder(300, channels, 'gat')).to(dev)
x, train_pos_edge_index = train_data.x.to(dev), train_data.pos_edge_label_index.to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 401):
    loss = train(epoch)
    auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)
    writer.add_scalar("AUC", auc, epoch)
    writer.add_scalar("AP", ap, epoch)
    if epoch % 10 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}, Loss: {:.4f}'.format(epoch, auc, ap, loss))

Epoch: 010, AUC: 0.8365, AP: 0.8324, Loss: 0.9607
Epoch: 020, AUC: 0.8705, AP: 0.8702, Loss: 0.8800
Epoch: 030, AUC: 0.8889, AP: 0.8913, Loss: 0.8373
Epoch: 040, AUC: 0.8964, AP: 0.9009, Loss: 0.8187
Epoch: 050, AUC: 0.9042, AP: 0.9101, Loss: 0.8082
Epoch: 060, AUC: 0.9072, AP: 0.9133, Loss: 0.7995
Epoch: 070, AUC: 0.9075, AP: 0.9147, Loss: 0.7924
Epoch: 080, AUC: 0.9079, AP: 0.9160, Loss: 0.7824
Epoch: 090, AUC: 0.9111, AP: 0.9193, Loss: 0.7835
Epoch: 100, AUC: 0.9116, AP: 0.9202, Loss: 0.7766
Epoch: 110, AUC: 0.9114, AP: 0.9205, Loss: 0.7768
Epoch: 120, AUC: 0.9131, AP: 0.9222, Loss: 0.7757
Epoch: 130, AUC: 0.9108, AP: 0.9207, Loss: 0.7678
Epoch: 140, AUC: 0.9125, AP: 0.9222, Loss: 0.7693
Epoch: 150, AUC: 0.9123, AP: 0.9225, Loss: 0.7706
Epoch: 160, AUC: 0.9128, AP: 0.9231, Loss: 0.7659
Epoch: 170, AUC: 0.9139, AP: 0.9244, Loss: 0.7657
Epoch: 180, AUC: 0.9141, AP: 0.9245, Loss: 0.7610
Epoch: 190, AUC: 0.9133, AP: 0.9243, Loss: 0.7626
Epoch: 200, AUC: 0.9118, AP: 0.9235, Loss: 0.7634


#### Examples

Let us see the nearest neighbours for the unseen words from the test set.

In [32]:
model.eval()
new_x = torch.tensor([fasttext[index_to_key[i]] for i in index_to_key], dtype=torch.float).to(dev)
z = model.encode(new_x, train_pos_edge_index)

In [33]:
id2syns = {}
syns2id = {}
with open('wordnet_n_is_directed_1_en_synsets/node') as f:
    for line in f:
        id2syns[line.split()[0]] = line.split()[-1]
        syns2id[line.split()[-1]] = line.split()[0]

In [34]:
par2orph = {}
orph2par = {}
with open('wordnet_n_is_directed_1_en_synsets/link') as f:
    for line in f:
        par_id = line.split()[0]
        child_id = line.split()[-1]
        
        if "ORPHAN_" in id2syns[child_id]:
            par2orph[id2syns[par_id]] = id2syns[child_id]
            orph2par[id2syns[child_id]] = id2syns[par_id]

In [35]:
c = 0
for word in fasttext.key_to_index:
    if ".n." not in word:
        cur_index = fasttext.key_to_index[word]
        tensor_ = torch.tensor([[cur_index]*(len(G.nodes)), [i for i in range(0, len(G.nodes))]])
        results = model.decode(z, tensor_)
        top10 = list(reversed(sorted([(index_to_key[i], round(float(score.cpu().detach().float()), 4)) for i, score in enumerate(results)], key=lambda x: x[1])))[:10]       
        print(orph2par[word], ":", top10)
        print("="*10)
        c += 1
        if c == 20:
            break

course.n.04 : [('act.n.03', 0.9975), ('way.n.10', 0.9955), ('way.n.09', 0.9955), ('way.n.05', 0.9952), ('ORPHAN_100000000', 0.9925), ('path.n.02', 0.9924), ('action.n.10', 0.9909), ('course.n.04', 0.9901), ('warpath.n.02', 0.9887), ('course.n.07', 0.9874)]
recovery.n.03 : [('act.n.03', 0.996), ('act.n.01', 0.99), ('legislative_act.n.01', 0.9802), ('rescue.n.01', 0.9785), ('healing.n.01', 0.9777), ('nay.n.01', 0.9747), ('ORPHAN_100000001', 0.9734), ('res_gestae.n.02', 0.9723), ('recovery.n.03', 0.9713), ('beatification.n.02', 0.9693)]
disappearance.n.01 : [('vanishing.n.01', 0.9848), ('ORPHAN_100000002', 0.9848), ('tracing.n.02', 0.9578), ('trace.n.06', 0.9506), ('trace.n.05', 0.9506), ('ORPHAN_100000445', 0.9481), ('flow.n.04', 0.9458), ('drawing.n.02', 0.9392), ('vanishing_point.n.01', 0.9344), ('silhouette.n.02', 0.933)]
hit.n.03 : [('hit.n.05', 0.9233), ('hit.n.01', 0.9233), ('win.n.01', 0.9094), ('loss.n.01', 0.878), ('financial_loss.n.01', 0.8629), ('fluke.n.02', 0.8623), ('base_h

# Graph Attention Networks v2 (GATv2)

This is a PyTorch implementation of the GATv2 operator from the paper How Attentive are Graph Attention Networks?.

https://nn.labml.ai/graphs/gatv2/index.html

## GraphBERT

https://github.com/jwzhanggy/Graph-Bert

Yet another model for embedding generation is GraphBert. Instead of feeding large input graph, we train GRAPH-BERT with sampled subgraphs within their local contexts. The input vector embeddings to be fed to the graphtransformer model actually cover four parts: (1) raw feature vector embedding, (2) Weisfeiler-Lehman absolute role embedding, (3) intimacy based relative positional embedding, and (4) hop based relative distance embedding, respectively.

GRAPH-BERT is trained with the node attribute reconstruction and structure recovery tasks.

![](https://github.com/jwzhanggy/Graph-Bert/raw/master/result/screenshot/model.png)

## Subgraph Sampling

![](https://i.ibb.co/5cbjJZ6/photo-2021-12-07-16-41-32.jpg)

## Positional embeddings

### Weisfeiler-Lehman Absolute Role Embedding

![](https://i.ibb.co/bgT7gqb/wl.png)

### Intimacy based Relative Positional Embedding

![](https://i.ibb.co/34FvCf0/photo-2021-12-07-16-52-30.jpg)

### Hop based Relative Distance Embedding
![](https://i.ibb.co/tCzRcfK/hops-drawio.png)

Actually, you are simply expected to run two scripts: `script_1_preprocess.py` and `script_2_pre_train.py`

In [36]:
!git clone https://github.com/jwzhanggy/Graph-Bert.git

Cloning into 'Graph-Bert'...
remote: Enumerating objects: 450, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 450 (delta 106), reused 79 (delta 78), pack-reused 314
Receiving objects: 100% (450/450), 2.23 MiB | 19.50 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [37]:
%cd Graph-Bert
!python3 script_1_preprocess.py

/content/Graph-Bert
************ Start ************
WL, dataset: cora
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 1
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 2
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 3
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 4
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 5
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 6
Loading cora dataset...
************ Finish ************
************ Start ************
Subgraph Batching, dataset: cora, k: 7
Loading cora dataset...
**********

## View and evaluate results

In [41]:
!gdown 1IAfd9tRgtVtdosM5vuDdxh-VSBFp3mzI
!gdown 1LItbxEcchOfU4TrlLBZjQweC8jpQ3b3Q
!gdown 1VLLLyu9YyLX3uCojiTm_VLtgK2gKCCfW
!gdown 1h5sSbFeCJbouH96fKIZDF2xugNiKf3La

Downloading...
From: https://drive.google.com/uc?id=1IAfd9tRgtVtdosM5vuDdxh-VSBFp3mzI
To: /content/Graph-Bert/MethodGraphBertGraphRecovery_model_test_embeddings.txt
100% 258k/258k [00:00<00:00, 105MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LItbxEcchOfU4TrlLBZjQweC8jpQ3b3Q
To: /content/Graph-Bert/MethodGraphBertGraphRecovery_model_train_embeddings_.txt
100% 196M/196M [00:03<00:00, 56.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VLLLyu9YyLX3uCojiTm_VLtgK2gKCCfW
To: /content/Graph-Bert/MethodGraphBertNodeConstruct_model_train_embeddings_.txt
100% 288M/288M [00:06<00:00, 42.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1h5sSbFeCJbouH96fKIZDF2xugNiKf3La
To: /content/Graph-Bert/MethodGraphBertNodeConstruct_model_test_embeddings.txt
100% 362k/362k [00:00<00:00, 111MB/s]


In [42]:
from gensim.models import KeyedVectors

In [43]:
graphBertNode_train = KeyedVectors.load_word2vec_format("MethodGraphBertNodeConstruct_model_train_embeddings_.txt")
graphBertNode_test = KeyedVectors.load_word2vec_format("MethodGraphBertNodeConstruct_model_test_embeddings.txt")

In [44]:
graphBertNode_train.similar_by_word("dog.n.01")

[('hound.n.01', 0.8980603814125061),
 ('working_dog.n.01', 0.8848254680633545),
 ('dandy.n.01', 0.8751208782196045),
 ('old_man.n.01', 0.8639864325523376),
 ('professional.n.01', 0.8605043888092041),
 ('gravida.n.02', 0.849956750869751),
 ('child.n.02', 0.8499069809913635),
 ('spaniel.n.01', 0.8490362167358398),
 ('subordinate.n.01', 0.8471304178237915),
 ('parent.n.01', 0.8452426195144653)]

In [45]:
fasttext.similar_by_word("dog.n.01")

[('dog.n.03', 0.9556826949119568),
 ('seizure-alert_dog.n.01', 0.9394533634185791),
 ('working_dog.n.01', 0.9265448451042175),
 ('dog_breeding.n.01', 0.9224575757980347),
 ('hunting_dog.n.01', 0.9179456233978271),
 ('dog_biscuit.n.01', 0.9155749678611755),
 ('ORPHAN_100000208', 0.9101989269256592),
 ('dog_catcher.n.01', 0.9075720906257629),
 ('raccoon_dog.n.01', 0.9067509770393372),
 ('dalmatian.n.02', 0.9053666591644287)]

In [48]:
wn.synset("dog.n.01").hyponyms()

[Synset('basenji.n.01'),
 Synset('corgi.n.01'),
 Synset('cur.n.01'),
 Synset('dalmatian.n.02'),
 Synset('great_pyrenees.n.01'),
 Synset('griffon.n.02'),
 Synset('hunting_dog.n.01'),
 Synset('lapdog.n.01'),
 Synset('leonberg.n.01'),
 Synset('mexican_hairless.n.01'),
 Synset('newfoundland.n.01'),
 Synset('pooch.n.01'),
 Synset('poodle.n.01'),
 Synset('pug.n.01'),
 Synset('puppy.n.01'),
 Synset('spitz.n.01'),
 Synset('toy_dog.n.01'),
 Synset('working_dog.n.01')]

# OpenHGNN library

In [49]:
pip install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.3 MB/s eta 0:00:00


In [51]:
pip install openhgnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.7/230.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openhgnn: filename=openhgnn-0.4.0-py3-none-any.whl size=297065 sha256=d77e2f138be3cd9b30dfa75456f8246ff03dc8e86bcce11e81b7f883145d3985
  Stored in directory: /root/.cache/pip/wheels/10/8a/b6/12b114efc5e310da3be61a15dd8aaf94c4e961f368cd78d1d6
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=cc1889c8e48c144eaa73c46e03772f7950e013746c903e72635d5e34fb

In [53]:
%cd ..

/content


In [56]:
!git clone https://github.com/BUPT-GAMMA/OpenHGNN

Cloning into 'OpenHGNN'...
remote: Enumerating objects: 6567, done.
remote: Counting objects: 100% (377/377), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 6567 (delta 245), reused 346 (delta 234), pack-reused 6190
Receiving objects: 100% (6567/6567), 15.20 MiB | 14.70 MiB/s, done.
Resolving deltas: 100% (4650/4650), done.


In [58]:
# from networkx.algorithms.centrality import edge_betweenness_centrality

In [60]:
!python ./OpenHGNN/main.py -m GTN -d imdb4GTN -t node_classification -g -1 --use_best_config

Load the best config of model: GTN for dataset: imdb4GTN.
------------------------------------------------------------------------------
 Basic setup of this experiment: 
     model: GTN    
     dataset: imdb4GTN   
     task: node_classification. 
 This experiment has following parameters. You can use set_params to edit them.
 Use print(experiment) to print this information again.
------------------------------------------------------------------------------
adaptive_lr_flag: True
dataset_name: imdb4GTN
device: cpu
dropout: 0.1
gpu: -1
hidden_dim: 128
hpo_search_space: None
hpo_trials: 100
identity: True
load_from_pretrained: False
lr: 0.01
max_epoch: 100
mini_batch_flag: False
model_name: GTN
norm_emd_flag: True
num_channels: 8
num_layers: 2
optimizer: Adam
out_dim: 16
output_dir: ./openhgnn/output/GTN
patience: 20
seed: 0
use_best_config: True
weight_decay: 0.001

11 Apr 12:48    INFO  [Config Info]	Model: GTN,	Task: node_classification,	Dataset: imdb4GTN
11 Apr 12:48    INFO  [NC 